In [ ]:
!git clone https://github.com/sskyisthelimit/DS_Internship2.git

In [2]:
cd DS_Internship2/task2/

/content/DS_Internship2/task2


In [ ]:
!pip install -r requirements.txt

In [4]:
cd src

/content/DS_Internship2/task2/src


In [20]:
!python -m inference --sentence "My friend has a cane corso." --device "cuda:0"

My: O
friend: O
has: O
a: O
cane: B-ANIMAL
corso: I-ANIMAL
.: O


I tried to make test set as real world as possible by picking

sentences from reddit r/Pets r/Animals

In [6]:
import json
# loading test dataset from repo
with open("/content/DS_Internship2/task2/dataset/processed/test_dataset.json", "r") as f:
  test_samples = json.load(f)

print(json.dumps(test_samples))

{"classes": ["B-ANIMAL", "I-ANIMAL"], "annotations": [["My dog is a chihuahua mix and she\u2019s 16 and is declining in health.", {"entities": [[3, 6, "B-ANIMAL"], [12, 21, "B-ANIMAL"]]}], ["She\u2019s on a chicken and rice diet because she was vomiting and it had blood in it.", {"entities": [[11, 18, "B-ANIMAL"]]}], ["She hasn\u2019t thrown up since I started the chicken and rice but I look at her and all I can think about is how she\u2019s in pain.", {"entities": [[41, 48, "B-ANIMAL"]]}], ["I do get angry when I see posts where the dog is actively dying and the owner is \"should I take him to the vet?", {"entities": [[42, 45, "B-ANIMAL"]]}], ["My Shih Tzu got diagnosed with kidney disease a year ago, and recently it worsened a lot.", {"entities": [[3, 7, "B-ANIMAL"], [8, 11, "I-ANIMAL"]]}], ["Pilot whales are the only animal with a higher brain-to-body mass ratio than humans, and they regularly commit mass suicide by beaching in the hundreds or thousands due to echolocation malfuncti

In [7]:
from inference import (model, tokenizer, post_process_tokens_and_labels,
                       predict_ner_labels, labels, label2id, id2label)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [8]:
import re
from utils import convert_to_bert_format

device = "cuda:0"
# Converting samples to same format as outputs of my model
# code at the end of src/utils.py
formatted_test_samples = []
for sample in test_samples["annotations"]:
  formatted_test_samples.append(
      convert_to_bert_format(sample[0], sample[1]["entities"], tokenizer))


In [9]:
print(formatted_test_samples[0][0])
print(formatted_test_samples[0][1])
print(formatted_test_samples[0][2])

['My', 'dog', 'is', 'a', 'ch', '##ih', '##ua', '##hua', 'mix', 'and', 'she', '’', 's', '16', 'and', 'is', 'declining', 'in', 'health', '.']
[0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
My dog is a chihuahua mix and she’s 16 and is declining in health.


In [ ]:
from sklearn.metrics import classification_report
import spacy
from spacy.tokens import Doc, Span
from spacy import displacy

nlp = spacy.blank("en")

def create_spacy_doc(tokens, labels, label_map):
    """
    Create a SpaCy Doc object with labeled entities for visualization.

    Args:
    tokens (list): List of tokens.
    labels (list): List of corresponding labels.
    label_map (dict): Mapping from label IDs to label names.

    Returns:
    Doc: A SpaCy Doc object with labeled entities.
    """
    doc = Doc(nlp.vocab, words=tokens)
    entities = []
    for i, label in enumerate(labels):
        if label > 0:  # Assuming 0 is 'O' or non-entity
            entities.append(Span(doc, i, i + 1, label=label_map[label]))
    doc.ents = entities
    return doc



def compute_metrics_and_visualize(formatted_test_samples, model, tokenizer, device, label_map):
    """
    Compute metrics and visualize problematic sentences.

    Args:
    formatted_test_samples (list): List of tuples containing tokens, labels, and sentences.
    model (torch.nn.Module): The fine-tuned NER model.
    tokenizer (transformers.PreTrainedTokenizer): The tokenizer used to tokenize the input.
    device (torch.device): The device (CPU or GPU) on which to run the model.
    label_map (dict): Mapping from label IDs to label names.
    """
    y_true, y_pred = [], []

    for tokens, labels, sentence in formatted_test_samples:
        # Predict labels for the sentence
        pred_tokens, pred_labels = predict_ner_labels(sentence, model, tokenizer, device, False)
        tokens, labels = post_process_tokens_and_labels(tokens, labels)

        # Adjust labels for equivalence rules
        adjusted_preds = [
            true if label == true else 0
            for label, true in zip(pred_labels, labels)
        ]

        y_true.extend(labels)
        y_pred.extend(adjusted_preds)

        # Check for problematic sentences
        if labels != adjusted_preds:
            print("Problematic Sentence:")
            print("Original:")
            displacy.render(create_spacy_doc(tokens, labels, label_map), style="ent", jupyter=True)
            print("Predicted:")
            displacy.render(create_spacy_doc(pred_tokens, adjusted_preds, label_map), style="ent", jupyter=True)

    # Compute metrics
    print("Classification Report:")
    print(classification_report(y_true, y_pred, target_names=list(label_map.values())))


compute_metrics_and_visualize(formatted_test_samples, model.to(device), tokenizer, device, id2label)


Problematic Sentence:
Original:


Predicted:


Problematic Sentence:
Original:


Predicted:


Problematic Sentence:
Original:


...

Problematic Sentence:
Original:


Predicted:


Problematic Sentence:
Original:


Predicted:


Classification Report:
              precision    recall  f1-score   support

           O       0.98      1.00      0.99       911
    B-ANIMAL       1.00      0.83      0.91        71
    I-ANIMAL       1.00      0.78      0.88        27

    accuracy                           0.98      1009
   macro avg       0.99      0.87      0.92      1009
weighted avg       0.98      0.98      0.98      1009



i know it seems like it's to much not readable resulting visualized sentences but.

the fact that model doesn't catch some specific niche names for animal breed is actually less disturbung that some grouped samples with False Negatives that lower recall (don't catch world dog cat etc. when there's some already animal breed detected in sentence), this problem occurs specifically because of errors that Llama produces - doesn't higlight some non-main passed animal name in sentence it generates (my suspicion)
work on prompts, temperature etc. can help.

the fact that model catches breeds of animals can be used to do search (by creating index on that column) over some database of species to map what animal this is like "cane corso" -> dog and then do check with cv classifier